In [278]:
import pandas as pd
import altair as alt
from copy import deepcopy

chart_dict = {
  "config": {"style": {"cell": {"stroke": "transparent"}}},
  "layer": [
    {
      "layer": [
        {
          "data": {"values": [{"x": 0}, {"x": 100}]},
          "mark": {
            "strokeDash": [55,55],
            "type": "rule",
            "strokeWidth": 6,
            "color": "whitesmoke",
          },
          "encoding": {
            "x": {"field": "x", "type": "quantitative", "axis": None}
          }
        },
        {
          "data": {"values": [{"xt": -50, "y": 0}]},
          "mark": {"type": "rule", "strokeWidth": 6, "color": "whitesmoke"},
          "encoding": {
            "x": {"field": "xt", "type": "quantitative", "axis": None}
          }
        }
      ]
    },
    {
      "layer": [
        {
          "mark": {"type": "point", "filled": True, "size": 2000, "opacity": 1},
          "encoding": {
            "color": {
              "field": "Team",
              "scale": {"domain": ["D", "A"], "range": ["#d22", "#22d"]},
              "legend": None
            }
          }
        },
        {
          "mark": {
            "type": "point",
            "color": "black",
            "filled": False,
            "size": 2000,
            "strokeWidth": 3
          }
        }
      ],
      "encoding": {
        "x": {
          "axis": None,
          "field": "x",
          "scale": {"domain": [0, 100]},
          "type": "quantitative"
        },
        "y": {
          "axis": None,
          "field": "y",
          "scale": {"domain": [20, 80]},
          "type": "quantitative"
        }
      }
    },
    {
      "mark": {
        "type": "text",
        "align": "center",
        "color": "whitesmoke",
        "dy": 1,
        "fontSize": 26,
        "fontWeight": "bolder"
      },
      "encoding": {
        "text": {"field": "Label", "type": "nominal"},
        "x": {"axis": None, "field": "x", "type": "quantitative"},
        "y": {"axis": None, "field": "y", "type": "quantitative"}
      }
    },
    {
      "data": {"values": [{"x": 33, "y":34}, {"x": 48, "y": 34}, {"x":63, "y": 34}]},
      "mark": {
          "type": "point", 
          "shape": "arrow", 
          "size":1500, 
          "filled":True, 
          "yOffset":-40, 
          "fill":"black", 
          "opacity":1.0
      },
      "encoding": {
        "x": {"field": "x", "type": "quantitative"},
        "y": {"field": "y", "type": "quantitative"}
      }
    },
    {
      "data": {"values": [{"x": -50, "y": 50, "img": "https://raw.githubusercontent.com/samnlindsay/RugbyResults/master/ball_icon.png"}]},
      "mark": {"type": "image", "width": 25, "height": 25},
      "encoding": {
        "x": {"field": "x", "type": "quantitative"},
        "y": {"field": "y", "type": "quantitative"},
        "url": {"field": "img", "type": "nominal"}
      }
}
  ],
  "background": "#6c6",
  "data": {
    "values": None
  },
  "height": 400,
  "width": 500,
  "$schema": "https://vega.github.io/schema/vega-lite/v5.17.0.json"
}


def chart_from_data(df, arrows=None, chart_dict=chart_dict, ball_pos=[-50,50], as_schema=False):
    schema = deepcopy(chart_dict)
    schema["data"]["values"] = df.to_dict(orient="records")
    schema["layer"][4]["data"]["values"][0]["x"] = ball_pos[0]
    schema["layer"][4]["data"]["values"][0]["y"] = ball_pos[1]
    if arrows is None:
      del schema["layer"][3]
    else:
      schema["layer"][3]["data"]["values"] = arrows.to_dict(orient="records")

    if as_schema:
      return schema
    else:
      return alt.Chart.from_dict(schema)



d_line_y = 56
a_line_y = 44
a_walkin_y = 34

min_x = 3
max_x = 95
mid_x = (min_x + max_x) / 2.0

a2 = (-52,50)
d2 = (-25,75)

a9 = (50,25)
d9 = (80,75)

setup_lookup = {
   "A": "Auckland",
   "C": "Crusaders",
   "H": "Highlanders",
   "W": "Waikato"
}

def spacing(type,n=7):
    if type == "even":
        return (max_x - min_x) / (n-1)
    elif type == "lift":
        return 6.0
    elif type == "prelift":
        return 10.0
    elif type == "tight":
        return 14.0
    
def defence(n=7):
    d_line = []
    d_line.append({"Team": "D", "x": d2[0], "y": d2[1], "Label": "", "Jumper": "", "Order": -1})
    d_line.append({"Team": "D", "x": d9[0], "y": d9[1], "Label": "", "Jumper": "", "Order": 9})

    for i in range(1,n+1):
        x = min_x + min([spacing("even",n),20]) * (i-1)
        d_line.append({"Team": "D", "x": x, "y": d_line_y, "Label": "", "Jumper": "", "Order": i})

    return pd.DataFrame(d_line)

In [291]:

# the data frame is comprised of the following columns:
# - Team: A or D
# - x: x-coordinate
# - y: y-coordinate
# - Label: J for Jumper, 9 for scrum half, else empty
# - Order: Order of the player in the formation (-1 for hooker, 9 for scrum half)
# - Jumper: Jumper number

# Initialise attack formation, before adjusting for setup etc.
def attack_init(n=7):
    a_line = [
        {"Team": "A", "x": a2[0], "y": a2[1], "Label": "2", "Jumper": "", "Order": -1},
        {"Team": "A", "x": a9[0], "y": a9[1], "Label": "9", "Jumper": "", "Order": 9},
    ]
    for i in range(1,n+1):
        x = min_x + min([spacing("even",n),20]) * (i-1)
        a_line.append({"Team": "A", "x": x, "y": a_line_y, "Label": "", "Jumper": "", "Order": i})
        
    return pd.DataFrame(a_line)
    

# Order index of players who walk into the line (these will have arrows attached to them)
def get_walk_ins(n, setup):
    if setup == "H":
        return list(range(2,n))
    elif setup == "C":
        if n > 5:
            return [n-4,n-3,n-2]
        else:
            return [2,3,4]
    elif setup == "W":
        if n > 5:
            return [2,n-3,n-1]
        else:
            return [2,4]
    elif setup == "A":
        return None
    
def get_arrows(df, walk_in):
    if walk_in is None:
        return None
    
    a = []
    for i in walk_in:
        x = df.loc[df["Order"]==i, "x"].values[0]
        a.append({"x": x, "y": a_walkin_y})
    return pd.DataFrame(a)

# Order index of jumpers (these will be labelled with "J")
def jumpers(n, setup):
    if setup == "H":
        if n > 5:
            return [2,n-3,n-1]
        else:
            return [2,3,4]
    elif setup == "C":
        if n == 7:
            return [2,4,n-1]
        elif n == 6:
            return [2, None, n-1]
        return [2,3,4]
    elif setup == "W":
        if n > 5:
            return [2,4,n-1]
        else:
            return [2,3,4]
    elif setup == "A":
        if n > 5:
            return [3,n-2,n-1]
        else:
            return [3,None,4]
    
def label_jumpers(df, jumpers):
    for i,o in enumerate(jumpers):
        df.loc[df["Order"]==o, "Label"] = "J"
        
        df.loc[df["Order"]==o, "Jumper"] = ["2", "3", "1"][i] #if len(jumpers)==3 else ["2", "1"]

    df["x"] = df["x"].astype(float)
    return df

# Move walk-ins to the walk-in line and return arrows    
def move_walkins(df, walk_in=None, a_walkin_y=a_walkin_y, n=7, setup="C"):
    df["x"] = df["x"].astype(float)
    
    if walk_in is None:
        return df
    else: 
        move = df["Order"].isin(walk_in) & df["Team"].eq("A")
        
        # Move y-coordinate of walk-ins
        df.loc[move, "y"] = a_walkin_y
        
        # Move x-coordinate of walk-ins (if 3 in a row)
        if walk_in == [min(walk_in) + a for a in [0,1,2]] and (setup != "H"):
            dx = spacing("prelift")
            x = float(df.loc[df["Order"].isin([1,n]), "x"].values.mean())
            df.loc[move, "x"] = [x-dx, x, x+dx]
            
        df["x"] = df["x"].astype(float)
        return df
    
# Adjust attack formation for setup
def adjust_spacing(df, n, setup):

    if setup == "C":
        dx = spacing("prelift")
        if n > 5:
            df.loc[df["Order"]==n-1, "x"] = max_x - dx
        if n==7:
            df.loc[df["Order"]==2, "x"] = min_x + dx
    elif setup == "W":
        if n > 5:
            df.loc[df["Order"]==4, "x"] = mid_x
        if n == 6:
            dx = spacing("tight")
            df.loc[df["Order"].isin([2,3,5]), "x"] = [mid_x-(2*dx), mid_x-dx, mid_x+dx]
    elif setup == "A":
        dx = spacing("tight")
        df.loc[df["Order"].isin(range(1,n-1)), "x"] = [min_x + i*dx for i in range(n-2)]
        if n > 5:
            df.loc[df["Order"]==n-1, "x"] = max_x - dx
        else:
            df.loc[df["Order"]==n-1, "x"] = mid_x
    
    df["x"] = df["x"].astype(float)
    return df    
    

def lineup(n=7, setup="H", receiver="9"):
    a = attack_init(n)
    walk = get_walk_ins(n, setup)
    jump = jumpers(n, setup)
    a = move_walkins(a, walk, n=n, setup=setup)
    a = label_jumpers(a, jump)
    a = adjust_spacing(a, n, setup)

    if setup == "A":
        a.loc[a["Order"]==2, "Label"] = "D"
    
    if receiver == "F":
        a.loc[a["Order"]==9, "Label"] = "+1"
    elif receiver is None:
        a = a[a["Order"] != 9]
    
    arrows = get_arrows(a, walk)
    return a, arrows

def setup_chart(n, setup, receiver="9"):
    df, arrows = lineup(n, setup, receiver)
    df = pd.concat([df, defence(n)])
    df["x"] = df["x"].astype(float)
    
    chart = chart_from_data(df, arrows)

    return chart

In [292]:
setup_chart(7, "W")

alt.LayerChart(...)

In [293]:
n = 7
setup = "H"
call = 1

df = lineup(n, setup)[0]

def setup_to_throw(df, n=7, setup="H", call=1):
    df.loc[df["y"]==a_walkin_y, "y"] = a_line_y 
    dx = spacing("lift")
    ddx = spacing("prelift")
    
    ###############
    # HIGHLANDERS #
    ###############
    if setup == "H":
        # Move players either side of Jumper {call}
        if call == 2:
            j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            x = df.loc[df["Order"]==(j-1), "x"].values[0]
            pod = [j-1, j, j+1]
            df.loc[df["Order"].isin(pod), "x"] = [x, x+dx, x+(2*dx)]
        else:
            j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            x = df.loc[df["Order"]==j, "x"].values[0]
            pod = [j-1, j, j+1]
            df.loc[df["Order"].isin(pod), "x"] = [x-dx, x, x+dx]
    
    ##############
    # CANTERBURY #
    ##############
    elif setup == "C":
        # Back pod
        j1 = df.loc[df["Jumper"]=="1", "Order"].values[0]
        if n > 5:
            x1 = df.loc[df["Order"]==j1, "x"].values[0]
        else:
            x1 = max_x - spacing("even", n)
        pod1 = [j1-1, j1, j1+1]

        # Front pod
        j2 = df.loc[df["Jumper"]=="2", "Order"].values[0]
        x2 = df.loc[df["Order"]==(j2-1), "x"].values[0]
        pod2 = [j2-1, j2, j2+1]

        dx1 = dx if call == 1 else ddx
        dx2 = dx if call == 2 else ddx

        if n > 5:
            df.loc[df["Order"].isin(pod1), "x"] = [x1-dx1, x1, x1+dx1]
            df.loc[df["Order"].isin(pod2), "x"] = [x2, x2+dx2, x2+(2*dx2)]
        else:
            if call == 1:   
                df.loc[df["Order"].isin(pod1), "x"] = [x1-dx1, x1, x1+dx1]
                df.loc[df["Order"].isin(pod2[:2]), "x"] = [x2, x2+dx2]
            elif call in [2,3]:
                df.loc[df["Order"].isin(pod1[1:]), "x"] = [x1, x1+dx1]
                df.loc[df["Order"].isin(pod2), "x"] = [x2, x2+dx2, x2+(2*dx2)]
        
        # Middle pod
        if call == 3:
            j3 = df.loc[df["Jumper"]==("3" if n!=6 else "1"), "Order"].values[0]
            pod3 = [1, j3, j3+1]
            x3 = mid_x if n > 5 else (min_x + df.loc[df["Order"]==4, "x"].values[0])/2
            df.loc[df["Order"].isin(pod3), "x"] = [mid_x-dx, mid_x, mid_x+dx]
        
    ############
    # AUCKLAND #
    ############
    elif setup == "A":        
        # Front jumper
        df.loc[df["Jumper"]=="2", "x"] = min_x + dx if call == 2 else min_x + ddx
        
        # Dummy jumper always moves out and back behind the front pod
        df.loc[df["Order"]==2, "x"] = (min_x + 2*dx + spacing("tight")) if call == 2 else mid_x

        if n == 7:
            # Middle man always supports J2
            df.loc[df["Order"]==4, "x"] = min_x + 2*ddx

    
        if call==2:
            j2 = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            pod2 = [1, j2, j2+1]
            df.loc[df["Order"].isin(pod2), "x"] = [min_x, min_x+dx, min_x+2*dx]
        elif call==3:
            j3 = df.loc[df["Jumper"]==("3" if n>5 else "1"), "Order"].values[0]
            pod3 = [2, j3, j3+1]
            df.loc[df["Order"].isin(pod3), "x"] = [mid_x-dx, mid_x, mid_x+dx]
        elif call==1:
            j1 = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            if n>5:
                x1 = df.loc[df["Order"]==j1, "x"].values[0]
                pod1 = [j1-1, j1, j1+1]
            else:
                x1 = df.loc[df["Order"]==n, "x"].values[0] - spacing("even", n)
                pod1 = [2, j1, n]
            df.loc[df["Order"].isin(pod1), "x"] = [x1-dx, x1, x1+dx]

    ###########
    # WAIKATO #
    ###########
    elif setup == "W":
        j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
        pod = [j-1, j, j+1]
        if call == 2:
            x = df.loc[df["Order"]==j-1, "x"].values[0]
            df.loc[df["Order"].isin(pod), "x"] = [x, x+dx, x+2*dx]
        else:
            x = df.loc[df["Order"]==j, "x"].values[0]
            if n==6 and call==1:
                x = max_x - spacing("even", n)
            df.loc[df["Order"].isin(pod), "x"] = [x-dx, x, x+dx]
    
    df["x"] = df["x"].astype(float)
    return df

def throw_chart(n, setup, call, receiver="9"):
    df, arrows = lineup(n, setup, receiver)
    df = setup_to_throw(df, n, setup, call)
    df = pd.concat([df, defence(n)])

    # df["x"] = df["x"].astype(float)

    if n==6 and setup=="C" and call==3:
        call = "1"
    if n==5 and setup=="A" and call==3:
        call = "1"
    ball_pos = [
        df.loc[df["Jumper"]==str(call), "x"].values[0] - 2,
        df.loc[df["Jumper"]==str(call), "y"].values[0] + 4,
    ]
    chart = chart_from_data(df, ball_pos=ball_pos)

    return chart

In [295]:
n=5
setup="C"
# display(setup_chart(n=n, setup=setup))
display(throw_chart(n=n, setup=setup, call=1))
display(throw_chart(n=n, setup=setup, call=2))
display(throw_chart(n=n, setup=setup, call=3))

/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/392985623.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]


alt.LayerChart(...)

/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/392985623.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]


alt.LayerChart(...)

/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/392985623.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]


alt.LayerChart(...)

In [290]:
for n in [5,6,7]:
    for setup in ["A", "C", "H", "W"]:
        print(n, setup)
        # Scrum Half
        chart = setup_chart(n, setup)
        chart.save(f"setup_{n}_{setup}.json", embed_options={"actions": False, "modebar": "none"})
        # No receiver
        chart = setup_chart(n, setup, receiver=None)
        chart.save(f"setup_{n}no9_{setup}.json", embed_options={"actions": False, "modebar": "none"})

        if n < 7:
            # No receiver
            chart = setup_chart(n, setup, receiver="F")
            chart.save(f"setup_{n}+1_{setup}.json", embed_options={"actions": False, "modebar": "none"})

        for call in [1,2,3]:
            print(call)
            # Scrum Half
            chart = throw_chart(n, setup, call)
            chart.save(f"throw_{n}_{setup}{call}.json", embed_options={"actions": False, "modebar": "none"})

            # No receiver
            chart = throw_chart(n, setup, call, receiver=None)
            chart.save(f"throw_{n}no9_{setup}{call}.json", embed_options={"actions": False, "modebar": "none"})

            if n < 7:
                chart = throw_chart(n, setup, call, receiver="F")
                chart.save(f"throw_{n}+1_{setup}{call}.json", embed_options={"actions": False, "modebar": "none"})

5 A
1
2
3
5 C


/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast

1


/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]


2


/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast

3


/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_45164/561406484.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.0, 43.0, 53.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[move, "x"] = [x-dx, x, x+dx]


5 H
1
2
3
5 W
1
2
3
6 A
1
2
3
6 C
1
2
3
6 H
1
2
3
6 W
1
2
3
7 A
1
2
3
7 C
1
2
3
7 H
1
2
3
7 W
1
2
3


In [47]:
setup_chart(7, "C", receiver=None)

alt.LayerChart(...)